# Malaria Detection

### Load Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import random, os
from skimage import transform
from pathlib import Path
from keras.preprocessing.image import load_img, img_to_array
from sklearn.utils import shuffle
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import Adam

In [ ]:
root_dir = 'cell_images'

In [ ]:
X = []
y = []

def load_images(name):
    img_dir = os.path.join(root_dir, name)
    for img in Path(img_dir).glob("*.png"):
        img = load_img(img)
        image_array = img_to_array(img)
        image_array = transform.resize(image_array, (50, 50, 3), anti_aliasing=True)
        X.append(image_array)
        if name == 'Parasitized':
            y.append(1)
        else:
            y.append(0)

In [ ]:
load_images('Parasitized')
load_images('Uninfected')

### Visualization

In [ ]:
def show_images(name): 
    img_dir = os.path.join(root_dir, name)
    plt.figure(figsize=(12,12))
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        random_img = random.choice([
            x for x in os.listdir(img_dir)
            if os.path.isfile(os.path.join(img_dir, x))
        ])
        img_normal = load_img(os.path.join(img_dir, random_img))
        ax.imshow(img_normal)
        plt.title(name)
    plt.show()

In [ ]:
show_images('Parasitized')
show_images('Uninfected')

### Preprocess

In [ ]:
X, y = shuffle(X, y)
X = np.array(X)
y = np.array(y)

In [ ]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2)

In [ ]:
plt.figure(figsize=(15,5))
n = 1
for d , name in zip([y_train , y_val , y_test] , ['Train', 'Validation', 'Test']):

    plt.subplot(1, 3, n)
    sns.countplot(x=d)
    plt.title(name)
    n+=1
plt.show()

In [ ]:
y_train = to_categorical(y_train, 2)
y_val = to_categorical(y_val, 2)
y_test = to_categorical(y_test, 2)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
validation_generator = train_datagen.flow(X_val, y_val, batch_size=32)
test_generator = test_datagen.flow(X_test, y_test, batch_size=32)

### Training

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(50, 50, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    validation_data=validation_generator, 
    validation_steps=validation_generator.n//validation_generator.batch_size,
    epochs=20)

### Evaluation

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
model.evaluate_generator(test_generator, steps=test_generator.n//test_generator.batch_size)

In [ ]:
'''
Inspiration
1. https://github.com/ushashwat/Chest-XRay-Pneumonia-Detection
'''